In [0]:
from pyspark.sql.functions import count, col, last
from pyspark.sql.window import Window

In [0]:
display(dbutils.fs.ls('/databricks-datasets/online_retail'))

path,name,size,modificationTime
dbfs:/databricks-datasets/online_retail/data-001/,data-001/,0,1725252383763


In [0]:
df=spark.read.csv('/databricks-datasets/online_retail/data-001/data.csv',header=True)


In [0]:
from pyspark.sql.functions import sum as sum_

null_counts = df.select(
    [sum_(col(c).isNull().cast("int")).alias(c) for c in df.columns]
)

display(null_counts)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,166,0,0,0,25281,0


In [0]:
df.filter(df.Description.isNull()).show()

+---------+---------+-----------+--------+-------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description|Quantity|  InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------+--------+-------------+---------+----------+--------------+
|   536414|    22139|       NULL|      56|12/1/10 11:52|        0|      NULL|United Kingdom|
|   536545|    21134|       NULL|       1|12/1/10 14:32|        0|      NULL|United Kingdom|
|   536546|    22145|       NULL|       1|12/1/10 14:33|        0|      NULL|United Kingdom|
|   536547|    37509|       NULL|       1|12/1/10 14:33|        0|      NULL|United Kingdom|
|   536549|   85226A|       NULL|       1|12/1/10 14:34|        0|      NULL|United Kingdom|
|   536550|    85044|       NULL|       1|12/1/10 14:34|        0|      NULL|United Kingdom|
|   536552|    20950|       NULL|       1|12/1/10 14:34|        0|      NULL|United Kingdom|
|   536553|    37461|       NULL|       3|12/1/10 14:35|        0|    

In [0]:
df.filter(df['Stockcode'] == '22139').select('Description').show()

+--------------------+
|         Description|
+--------------------+
|RETROSPOT TEA SET...|
|                NULL|
|              amazon|
|RETROSPOT TEA SET...|
|                NULL|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|                NULL|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|                NULL|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
+--------------------+
only showing top 20 rows



In [0]:
#counting distinct column to map description of null values using stock code
print("Description distinct count:", df.select(col('Description')).distinct().count())
print("stockcode distinct count:", df.select(col('StockCode')).distinct().count())


Description distinct count: 2914
stockcode distinct count: 2973


In [0]:
window_spec = Window.partitionBy('Stockcode').orderBy('Stockcode').rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Fill NaN values in 'description' with the last non-null value within the same 'stockcode'
df = df.withColumn('Description', last('Description', True).over(window_spec))


In [0]:
null_counts = df.select(
    [sum_(col(c).isNull().cast("int")).alias(c) for c in df.columns]
)

display(null_counts)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,47,0,0,0,25281,0


In [0]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
df.filter(df['Stockcode'] == '22139').select('Description').show()

+--------------------+
|         Description|
+--------------------+
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|              amazon|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
|RETROSPOT TEA SET...|
+--------------------+
only showing top 20 rows



In [0]:
for i in df:
    row_count= df.select(i).count()
    print(i,":", row_count) 

Column<'InvoiceNo'> : 65499
Column<'StockCode'> : 65499
Column<'Description'> : 65499
Column<'Quantity'> : 65499
Column<'InvoiceDate'> : 65499
Column<'UnitPrice'> : 65499
Column<'CustomerID'> : 65499
Column<'Country'> : 65499


In [0]:
df1= df.select('InvoiceNo', 'Description', 'Quantity', 'UnitPrice', 'StockCode')


In [0]:
# Adding new column as TotalPrice
df1 = df1.withColumn(
    'TotalPrice', 
    (df1.Quantity * df1.UnitPrice).cast('Float')
).withColumn('Quantity', df1.Quantity.cast('int'))

# Display the DataFrame
display(df1)

InvoiceNo,Description,Quantity,UnitPrice,StockCode,TotalPrice
536370,INFLATABLE POLITICAL GLOBE,48,0.85,10002,40.8
536382,INFLATABLE POLITICAL GLOBE,12,0.85,10002,10.2
536756,INFLATABLE POLITICAL GLOBE,1,0.85,10002,0.85
536863,INFLATABLE POLITICAL GLOBE,1,0.85,10002,0.85
536865,INFLATABLE POLITICAL GLOBE,5,1.66,10002,8.3
536876,INFLATABLE POLITICAL GLOBE,2,1.66,10002,3.32
537047,INFLATABLE POLITICAL GLOBE,1,0.85,10002,0.85
537227,INFLATABLE POLITICAL GLOBE,24,0.85,10002,20.4
537240,INFLATABLE POLITICAL GLOBE,1,1.66,10002,1.66
537638,INFLATABLE POLITICAL GLOBE,5,1.66,10002,8.3


In [0]:
#rename column
df1.select('Description', 'Quantity').withColumnRenamed('Description', 'Desc').groupBy('Desc').sum('Quantity').orderBy('sum(Quantity)', ascending=False).show()

+--------------------+-------------+
|                Desc|sum(Quantity)|
+--------------------+-------------+
|WHITE HANGING HEA...|         8189|
|PACK OF 72 RETROS...|         6634|
|WORLD WAR 2 GLIDE...|         6109|
|HAND WARMER BABUS...|         4060|
|HEART OF WICKER S...|         4047|
|JUMBO BAG RED RET...|         3993|
|SMALL POPCORN HOLDER|         3971|
|MINI PAINT SET VI...|         3615|
|ASSORTED COLOUR B...|         3374|
|FAIRY CAKE FLANNE...|         3365|
|GROW A FLYTRAP OR...|         2953|
|PACK OF 12 LONDON...|         2950|
| BROCADE RING PURSE |         2854|
|       CHILLI LIGHTS|         2786|
|CHARLOTTE BAG SUK...|         2722|
|ANTIQUE SILVER TE...|         2701|
|GIN + TONIC DIET ...|         2693|
|REGENCY CAKESTAND...|         2686|
|RED  HARMONICA IN...|         2669|
|JAM MAKING SET PR...|         2654|
+--------------------+-------------+
only showing top 20 rows

